# Lambda 3 - Filter Confidences

In [1]:
import sagemaker
import boto3
import os


bucket= sagemaker.session.Session().default_bucket()
role = sagemaker.get_execution_role()
region = boto3.client('sagemaker').meta.region_name

os.environ["DEFAULT_S3_BUCKET"] = bucket
os.environ["PREFIX"] = 'Udacity-AWSML-PRO2'

prefix = os.environ["PREFIX"]

#### From the previous lambda the output is...

In [2]:
event = {
 'statusCode': 200,
 'body': '{"inferences": [0.9969943761825562, 0.0030055944807827473]}'
}

In [7]:
import json

THRESHOLD = .93


def lambda_handler(event, context):

    # Grab the inferences from the event
    json_string = event['body']
    json_object = json.loads(json_string)
    inferences = json_object['inferences']
    

    # Check if any values in our inferences are above THRESHOLD
    meets_threshold = [1.0*(cls > THRESHOLD) for cls in inferences] 

    
    # If our threshold is met, pass our data back out of the
    # Step Function, else, end the Step Function with an error
    event = dict()
    if sum(meets_threshold):
        event["inferences"] = meets_threshold      
    else:
        raise("THRESHOLD_CONFIDENCE_NOT_MET")
    return {
        'statusCode': 200,
        'body': event #json.dumps(event)
    }

In [8]:
response_threshold = lambda_handler(event=event, context={'ContentType':'application/json'}) 
response_threshold

{'statusCode': 200, 'body': {'inferences': [1.0, 0.0]}}

###### Helper Function

##### 1.1 - Serialize Image data

In [83]:
%%writefile lambda_function.py

import json

THRESHOLD = .93


def lambda_handler(event, context):

    # Grab the inferences from the event
    json_string = event['body']
    json_object = json.loads(json_string)
    inferences = json_object['inferences']
    

    # Check if any values in our inferences are above THRESHOLD
    meets_threshold = [1.0*(cls > THRESHOLD) for cls in inferences] 

    
    # If our threshold is met, pass our data back out of the
    # Step Function, else, end the Step Function with an error
    event = dict()
    if sum(meets_threshold):
        event["inferences"] = meets_threshold      
    else:
        raise("THRESHOLD_CONFIDENCE_NOT_MET")
    return {
        'statusCode': 200,
        'body': event #json.dumps(event)
    }

Overwriting lambda_function.py


###### 1.2 - Zip to a File

In [84]:
from zipfile import ZipFile

with ZipFile('code.zip', 'w') as f:
    f.write('lambda_function.py')

###### 1.3 - Upload to S3

In [85]:
#upload_file_to_s3(file_name='code.zip', bucket=bucket, s3_prefix=prefix)

#### 1.4 - Create Lambda Function over AWS

In [86]:
lambda_client = boto3.client('lambda')

os.path
with open('code.zip', 'rb') as f:
    b_code = f.read()

In [87]:
response = lambda_client.create_function(
    FunctionName='filterConfidences',
    Runtime='python3.8',
    Handler='lambda_function.lambda_handler',
    Code={
        'ZipFile': b_code,
    },
    Description='string',
    Timeout=30,
    MemorySize=1024,
    Publish=True,
    PackageType='Zip',
    Role=role
)

In [88]:
response

{'ResponseMetadata': {'RequestId': 'f179231b-650a-477d-8bb2-761edfa07cb6',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Sun, 18 Sep 2022 03:59:13 GMT',
   'content-type': 'application/json',
   'content-length': '1035',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f179231b-650a-477d-8bb2-761edfa07cb6'},
  'RetryAttempts': 0},
 'FunctionName': 'filterConfidences',
 'FunctionArn': 'arn:aws:lambda:us-east-1:254050731868:function:filterConfidences',
 'Runtime': 'python3.8',
 'Role': 'arn:aws:iam::254050731868:role/service-role/AmazonSageMaker-ExecutionRole-20220428T195259',
 'Handler': 'lambda_function.lambda_handler',
 'CodeSize': 938,
 'Description': 'string',
 'Timeout': 30,
 'MemorySize': 1024,
 'LastModified': '2022-09-18T03:59:13.392+0000',
 'CodeSha256': '531T+Tdw5YIe95FAvIf8VOjYC7JR7qkY01Xh2eVnlYI=',
 'Version': '1',
 'TracingConfig': {'Mode': 'PassThrough'},
 'RevisionId': '377c70c9-90c0-41e8-b6af-cb2300c8be4f',
 'State': 'Pending',
 'StateReason': 'The function i

#### 1.5 - Invoke Endpoint

In [89]:
event = {
 'statusCode': 200,
 'body': '{"inferences": [0.9969943761825562, 0.0030055944807827473]}'
}

In [90]:
import json

lambda_client = boto3.client('lambda')
payload = event

payload_bytes = json.dumps(payload).encode('utf-8')
response = lambda_client.invoke(FunctionName='filterConfidences', InvocationType='Event', Payload=payload_bytes)

In [91]:
response

{'ResponseMetadata': {'RequestId': '17fa61f1-9f1e-4cc5-b287-ef68c9ab8c3d',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sun, 18 Sep 2022 03:59:57 GMT',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': '17fa61f1-9f1e-4cc5-b287-ef68c9ab8c3d',
   'x-amzn-remapped-content-length': '0',
   'x-amzn-trace-id': 'root=1-632697bc-136178f73a9f8feb0e53baaa;sampled=0'},
  'RetryAttempts': 0},
 'StatusCode': 202,
 'Payload': <botocore.response.StreamingBody at 0x7ff502607340>}